In [1]:
import warnings
warnings.filterwarnings("ignore")
import joblib
import gc
import multiprocessing as mp
import numpy as np
import pandas as pd
import _pickle as cpickle
import seaborn as sns
import pickle
import lightgbm as lgb
from lightgbm.sklearn import LGBMClassifier
import seaborn as sns
import matplotlib.pyplot as plt
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, f1_score,roc_curve
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, classification_report
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc

In [2]:
# Importing features

user_df = pd.read_pickle(r'C:\Sanal\data_science_dreamjob\portfolio project\instacart-market-basket-analysis\user_df.pkl')
prd_df = pd.read_pickle(r'C:\Sanal\data_science_dreamjob\portfolio project\instacart-market-basket-analysis\prd_df.pkl')
user_x_product = pd.read_pickle(r'C:\Sanal\data_science_dreamjob\portfolio project\instacart-market-basket-analysis\user_x_product.pkl')

In [3]:
# importing csv files
orderprdtrain_df = pd.read_csv(r'C:\Sanal\data_science_dreamjob\portfolio project\instacart-market-basket-analysis\order_products__train.csv')
orders_df = pd.read_csv(r'C:\Sanal\data_science_dreamjob\portfolio project\instacart-market-basket-analysis\orders.csv')

In [4]:
orders_df['eval_set'].value_counts()

prior    3214874
train     131209
test       75000
Name: eval_set, dtype: int64

In [5]:
# feature filesb
user_df.head()

order_number                   max_day_of_week max_order_hour_of_day  \
                 min max nanmean count                                         
user_id                                                                        
1                  1  10     6.0    59               4                     4   
2                  1  14     8.0   195               2                     2   
3                  1  12     6.0    88               0                     0   
4                  1   5     3.0    18               4                     4   
5                  1   4     2.0    37               3                     3   

        sum_days_since_prior_order mean_days_since_prior_order  \
                                                                 
user_id                                                          
1                           1149.5                   19.483051   
2                           3050.3                   15.642564   
3                           1007.0                   11.443182   
4                            259.4                   14.411111   
5                            499.1                   13.489189   

        reordered_ratio total_num_orders user_total_prods user_unique_prods  \
                                                                              
user_id                                                                       
1              0.694915              343               59                18   
2              0.476923             1567              195               102   
3              0.625000              528               88                33   
4              0.055556               52               18                17   
5              0.378378               92               37                23   

        user_average_basket user_order_starts_at avg_no_prds_each_purchase  \
                                                                             
user_id                                                                      
1                  0.172012               431534                  5.900000   
2                  0.124442               738281                 13.928571   
3                  0.166667               444309                  7.333333   
4                  0.346154                94891                  3.600000   
5                  0.402174               157374                  9.250000   

        median_no_prds_each_purchase min_no_prds_each_purchase  \
                                                                 
user_id                                                          
1                                5.5                         4   
2                               13.5                         5   
3                                7.0                         5   
4                                3.0                         2   
5                               10.0                         5   

        max_no_prds_each_purchase  
                                   
user_id                            
1                               9  
2                              26  
3                              11  
4                               7  
5                              12

In [6]:
prd_df.head()

,product_id,times_purchased,prod_tot_reorders,prod_reordered_ratio,avg_cart_position,median_cart_position,sum_cart_position,min_cart_position,max_cart_position,std_cart_position,aisle_reorder_ratio,department_reorder_ratio
0,1,1852,1136,0.613391,5.801836,4.0,10745,1,74,5.575389,0.548698,0.574180
1,2,90,12,0.133333,9.888889,8.0,890,1,45,7.821670,0.152391,0.346721
2,3,277,203,0.732852,6.415162,4.0,1777,1,41,6.472701,0.527615,0.653460
3,4,329,147,0.446809,9.507599,8.0,3128,1,36,6.861485,0.556655,0.541885
4,5,15,9,0.600000,6.466667,6.0,97,3,16,3.563038,0.280627,0.346721


In [7]:
user_x_product.head()

,user_id,product_id,times_brought,uxp_total_reordered,uxp_first_order_num,uxp_avg_cart_pos,uxp_min_cart_pos,uxp_max_cart_pos,uxp_median_cart_pos,uxp_last_order_num,uxp_avg_dow,uxp_median_dow,uxp_avg_hour_of_the_day,uxp_median_hour_of_the_day,uxp_order_rate,uxp_sum_cart_pos,uxp_min_order_days,uxp_max_order_days,uxp_median_order_days,uxp_reorder_ratio
0,1,196,10,9,1,1.400000,1,4,1.0,10,2.500000,2.5,10.300000,8.5,1.0,14,0.0,30.0,19.5,0.900000
1,1,10258,9,8,2,3.333333,2,6,3.0,10,2.555556,3.0,10.555556,9.0,0.9,30,0.0,30.0,20.0,0.888889
2,1,10326,1,0,5,5.000000,5,5,5.0,5,4.000000,4.0,15.000000,15.0,0.1,5,28.0,28.0,28.0,0.000000
3,1,12427,10,9,1,3.300000,1,9,2.5,10,2.500000,2.5,10.300000,8.5,1.0,33,0.0,30.0,19.5,0.900000
4,1,13032,3,2,2,6.333333,5,8,6.0,10,2.666667,3.0,8.000000,8.0,0.3,19,15.0,30.0,20.0,0.666667


# Merging products, users, user-product dataframes

In [8]:
# Merging user_x_product features with users features
data = user_x_product.merge(user_df, on='user_id', how='left')
data.head()

,user_id,product_id,times_brought,uxp_total_reordered,uxp_first_order_num,uxp_avg_cart_pos,uxp_min_cart_pos,uxp_max_cart_pos,uxp_median_cart_pos,uxp_last_order_num,...,"(reordered_ratio, )","(total_num_orders, )","(user_total_prods, )","(user_unique_prods, )","(user_average_basket, )","(user_order_starts_at, )","(avg_no_prds_each_purchase, )","(median_no_prds_each_purchase, )","(min_no_prds_each_purchase, )","(max_no_prds_each_purchase, )"
0,1,196,10,9,1,1.400000,1,4,1.0,10,...,0.694915,343,59,18,0.172012,431534,5.9,5.5,4,9
1,1,10258,9,8,2,3.333333,2,6,3.0,10,...,0.694915,343,59,18,0.172012,431534,5.9,5.5,4,9
2,1,10326,1,0,5,5.000000,5,5,5.0,5,...,0.694915,343,59,18,0.172012,431534,5.9,5.5,4,9
3,1,12427,10,9,1,3.300000,1,9,2.5,10,...,0.694915,343,59,18,0.172012,431534,5.9,5.5,4,9
4,1,13032,3,2,2,6.333333,5,8,6.0,10,...,0.694915,343,59,18,0.172012,431534,5.9,5.5,4,9


In [9]:
# Merging product features with data.
data = data.merge(prd_df, on='product_id', how='left')
data.head()

,user_id,product_id,times_brought,uxp_total_reordered,uxp_first_order_num,uxp_avg_cart_pos,uxp_min_cart_pos,uxp_max_cart_pos,uxp_median_cart_pos,uxp_last_order_num,...,prod_tot_reorders,prod_reordered_ratio,avg_cart_position,median_cart_position,sum_cart_position,min_cart_position,max_cart_position,std_cart_position,aisle_reorder_ratio,department_reorder_ratio
0,1,196,10,9,1,1.400000,1,4,1.0,10,...,27791,0.776480,3.721774,2.0,133206,1,60,4.110813,0.638832,0.653460
1,1,10258,9,8,2,3.333333,2,6,3.0,10,...,1389,0.713772,4.277492,3.0,8324,1,29,3.567502,0.519170,0.574180
2,1,10326,1,0,5,5.000000,5,5,5.0,5,...,3603,0.652009,4.191097,3.0,23160,1,37,3.611700,0.718104,0.649913
3,1,12427,10,9,1,3.300000,1,9,2.5,10,...,4797,0.740735,4.760037,3.0,30826,1,61,4.782450,0.591986,0.574180
4,1,13032,3,2,2,6.333333,5,8,6.0,10,...,2465,0.657158,5.622767,4.0,21091,1,66,5.345184,0.571584,0.560922


In [10]:
data.shape

(13307953, 49)

# Creating Train and Test Datasets

In [11]:
# Orders data
orders_df.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [12]:
# future orders (train and test under eval_set)
future_orders = orders_df[((orders_df['eval_set'] == 'train') | (orders_df['eval_set'] == 'test'))]
future_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
10,1187899,1,train,11,4,8,14.0
25,1492625,2,train,15,1,11,30.0
38,2774568,3,test,13,5,15,11.0
44,329954,4,test,6,3,12,30.0
49,2196797,5,train,5,0,11,6.0


In [13]:
future_orders = future_orders[['user_id', 'order_id', 'eval_set']]
future_orders.head()
future_orders.shape

(206209, 3)

In [14]:
#merging data with future orders
data = data.merge(future_orders, on='user_id', how='left')
data.head()

,user_id,product_id,times_brought,uxp_total_reordered,uxp_first_order_num,uxp_avg_cart_pos,uxp_min_cart_pos,uxp_max_cart_pos,uxp_median_cart_pos,uxp_last_order_num,...,avg_cart_position,median_cart_position,sum_cart_position,min_cart_position,max_cart_position,std_cart_position,aisle_reorder_ratio,department_reorder_ratio,order_id,eval_set
0,1,196,10,9,1,1.400000,1,4,1.0,10,...,3.721774,2.0,133206,1,60,4.110813,0.638832,0.653460,1187899,train
1,1,10258,9,8,2,3.333333,2,6,3.0,10,...,4.277492,3.0,8324,1,29,3.567502,0.519170,0.574180,1187899,train
2,1,10326,1,0,5,5.000000,5,5,5.0,5,...,4.191097,3.0,23160,1,37,3.611700,0.718104,0.649913,1187899,train
3,1,12427,10,9,1,3.300000,1,9,2.5,10,...,4.760037,3.0,30826,1,61,4.782450,0.591986,0.574180,1187899,train
4,1,13032,3,2,2,6.333333,5,8,6.0,10,...,5.622767,4.0,21091,1,66,5.345184,0.571584,0.560922,1187899,train


In [15]:
data['eval_set'].value_counts()

train    8474661
test     4833292
Name: eval_set, dtype: int64

In [16]:
#Getting only the training data set
data_train = data[data['eval_set'] == 'train']
data_train.head()

,user_id,product_id,times_brought,uxp_total_reordered,uxp_first_order_num,uxp_avg_cart_pos,uxp_min_cart_pos,uxp_max_cart_pos,uxp_median_cart_pos,uxp_last_order_num,...,avg_cart_position,median_cart_position,sum_cart_position,min_cart_position,max_cart_position,std_cart_position,aisle_reorder_ratio,department_reorder_ratio,order_id,eval_set
0,1,196,10,9,1,1.400000,1,4,1.0,10,...,3.721774,2.0,133206,1,60,4.110813,0.638832,0.653460,1187899,train
1,1,10258,9,8,2,3.333333,2,6,3.0,10,...,4.277492,3.0,8324,1,29,3.567502,0.519170,0.574180,1187899,train
2,1,10326,1,0,5,5.000000,5,5,5.0,5,...,4.191097,3.0,23160,1,37,3.611700,0.718104,0.649913,1187899,train
3,1,12427,10,9,1,3.300000,1,9,2.5,10,...,4.760037,3.0,30826,1,61,4.782450,0.591986,0.574180,1187899,train
4,1,13032,3,2,2,6.333333,5,8,6.0,10,...,5.622767,4.0,21091,1,66,5.345184,0.571584,0.560922,1187899,train


In [17]:
# Merging the information contained in order_products_train csv with data_train
data_train = data_train.merge(orderprdtrain_df[['product_id','order_id','reordered']], on=['product_id','order_id'],how='left')
data_train.head()

,user_id,product_id,times_brought,uxp_total_reordered,uxp_first_order_num,uxp_avg_cart_pos,uxp_min_cart_pos,uxp_max_cart_pos,uxp_median_cart_pos,uxp_last_order_num,...,median_cart_position,sum_cart_position,min_cart_position,max_cart_position,std_cart_position,aisle_reorder_ratio,department_reorder_ratio,order_id,eval_set,reordered
0,1,196,10,9,1,1.400000,1,4,1.0,10,...,2.0,133206,1,60,4.110813,0.638832,0.653460,1187899,train,1.0
1,1,10258,9,8,2,3.333333,2,6,3.0,10,...,3.0,8324,1,29,3.567502,0.519170,0.574180,1187899,train,1.0
2,1,10326,1,0,5,5.000000,5,5,5.0,5,...,3.0,23160,1,37,3.611700,0.718104,0.649913,1187899,train,NaN
3,1,12427,10,9,1,3.300000,1,9,2.5,10,...,3.0,30826,1,61,4.782450,0.591986,0.574180,1187899,train,NaN
4,1,13032,3,2,2,6.333333,5,8,6.0,10,...,4.0,21091,1,66,5.345184,0.571584,0.560922,1187899,train,1.0


In [18]:
# replace Nan with a 0 in data_train
data_train['reordered'].fillna(0, inplace=True)

In [19]:
#setting the index for training data as a combination of user_id and product_id
data_train = data_train.set_index(['user_id', 'product_id'])

#Dropping the eval_set and order_id columns
data_train.drop(['eval_set', 'order_id'], axis=1, inplace=True)

display
data_train.head()

times_brought  uxp_total_reordered  uxp_first_order_num  \
user_id product_id                                                            
1       196                    10                    9                    1   
        10258                   9                    8                    2   
        10326                   1                    0                    5   
        12427                  10                    9                    1   
        13032                   3                    2                    2   

                    uxp_avg_cart_pos  uxp_min_cart_pos  uxp_max_cart_pos  \
user_id product_id                                                         
1       196                 1.400000                 1                 4   
        10258               3.333333                 2                 6   
        10326               5.000000                 5                 5   
        12427               3.300000                 1                 9   
        13032               6.333333                 5                 8   

                    uxp_median_cart_pos  uxp_last_order_num  uxp_avg_dow  \
user_id product_id                                                         
1       196                         1.0                  10     2.500000   
        10258                       3.0                  10     2.555556   
        10326                       5.0                   5     4.000000   
        12427                       2.5                  10     2.500000   
        13032                       6.0                  10     2.666667   

                    uxp_median_dow  ...  prod_reordered_ratio  \
user_id product_id                  ...                         
1       196                    2.5  ...              0.776480   
        10258                  3.0  ...              0.713772   
        10326                  4.0  ...              0.652009   
        12427                  2.5  ...              0.740735   
        13032                  3.0  ...              0.657158   

                    avg_cart_position  median_cart_position  \
user_id product_id                                            
1       196                  3.721774                   2.0   
        10258                4.277492                   3.0   
        10326                4.191097                   3.0   
        12427                4.760037                   3.0   
        13032                5.622767                   4.0   

                    sum_cart_position  min_cart_position  max_cart_position  \
user_id product_id                                                            
1       196                    133206                  1                 60   
        10258                    8324                  1                 29   
        10326                   23160                  1                 37   
        12427                   30826                  1                 61   
        13032                   21091                  1                 66   

                    std_cart_position  aisle_reorder_ratio  \
user_id product_id                                           
1       196                  4.110813             0.638832   
        10258                3.567502             0.519170   
        10326                3.611700             0.718104   
        12427                4.782450             0.591986   
        13032                5.345184             0.571584   

                    department_reorder_ratio  reordered  
user_id product_id                                       
1       196                         0.653460        1.0  
        10258                       0.574180        1.0  
        10326                       0.649913        0.0  
        12427                       0.574180        0.0  
        13032                       0.560922        1.0  

[5 rows x 48 columns]

In [20]:
# following same steps as above for test data

data_test = data[data['eval_set'] == 'test']

# Setting the index for train data as a combination of user-id and product-id
data_test = data_test.set_index(['user_id','product_id'])

# Dropping the eval_set and order_id columns
data_test.drop(['eval_set','order_id'],axis=1,inplace=True)

#display
data_test.head()

times_brought  uxp_total_reordered  uxp_first_order_num  \
user_id product_id                                                            
3       248                     1                    0                    2   
        1005                    1                    0                   10   
        1819                    3                    2                    4   
        7503                    1                    0                    3   
        8021                    1                    0                    2   

                    uxp_avg_cart_pos  uxp_min_cart_pos  uxp_max_cart_pos  \
user_id product_id                                                         
3       248                 3.000000                 3                 3   
        1005                5.000000                 5                 5   
        1819                2.666667                 1                 5   
        7503                6.000000                 6                 6   
        8021                5.000000                 5                 5   

                    uxp_median_cart_pos  uxp_last_order_num  uxp_avg_dow  \
user_id product_id                                                         
3       248                         3.0                   2     3.000000   
        1005                        5.0                  10     3.000000   
        1819                        2.0                   7     0.666667   
        7503                        6.0                   3     3.000000   
        8021                        5.0                   2     3.000000   

                    uxp_median_dow  ...  prod_tot_reorders  \
user_id product_id                  ...                      
3       248                    3.0  ...               2550   
        1005                   3.0  ...                204   
        1819                   0.0  ...               1193   
        7503                   3.0  ...               6905   
        8021                   3.0  ...              16472   

                    prod_reordered_ratio  avg_cart_position  \
user_id product_id                                            
3       248                     0.400251          10.620782   
        1005                    0.440605           9.498920   
        1819                    0.492162           9.287541   
        7503                    0.553551           9.547379   
        8021                    0.591157           8.822854   

                    median_cart_position  sum_cart_position  \
user_id product_id                                            
3       248                          9.0              67665   
        1005                         8.0               4398   
        1819                         7.0              22513   
        7503                         8.0             119094   
        8021                         7.0             245840   

                    min_cart_position  max_cart_position  std_cart_position  \
user_id product_id                                                            
3       248                         1                 91           8.025390   
        1005                        1                 74           7.984800   
        1819                        1                 68           7.441465   
        7503                        1                 79           7.622521   
        8021                        1                 90           7.211007   

                    aisle_reorder_ratio  department_reorder_ratio  
user_id product_id                                                 
3       248                    0.519170                  0.574180  
        1005                   0.527615                  0.653460  
        1819                   0.487633                  0.346721  
        7503                   0.519170                  0.574180  
        8021                   0.528005                  0.402178  

[5 rows x 47 columns]

In [21]:
#shape of train and test data
print('Train data shape -',data_train.shape)
print('Test data shape -',data_test.shape)

Train data shape - (8474661, 48)
Test data shape - (4833292, 47)


In [22]:
data_train.head()

times_brought  uxp_total_reordered  uxp_first_order_num  \
user_id product_id                                                            
1       196                    10                    9                    1   
        10258                   9                    8                    2   
        10326                   1                    0                    5   
        12427                  10                    9                    1   
        13032                   3                    2                    2   

                    uxp_avg_cart_pos  uxp_min_cart_pos  uxp_max_cart_pos  \
user_id product_id                                                         
1       196                 1.400000                 1                 4   
        10258               3.333333                 2                 6   
        10326               5.000000                 5                 5   
        12427               3.300000                 1                 9   
        13032               6.333333                 5                 8   

                    uxp_median_cart_pos  uxp_last_order_num  uxp_avg_dow  \
user_id product_id                                                         
1       196                         1.0                  10     2.500000   
        10258                       3.0                  10     2.555556   
        10326                       5.0                   5     4.000000   
        12427                       2.5                  10     2.500000   
        13032                       6.0                  10     2.666667   

                    uxp_median_dow  ...  prod_reordered_ratio  \
user_id product_id                  ...                         
1       196                    2.5  ...              0.776480   
        10258                  3.0  ...              0.713772   
        10326                  4.0  ...              0.652009   
        12427                  2.5  ...              0.740735   
        13032                  3.0  ...              0.657158   

                    avg_cart_position  median_cart_position  \
user_id product_id                                            
1       196                  3.721774                   2.0   
        10258                4.277492                   3.0   
        10326                4.191097                   3.0   
        12427                4.760037                   3.0   
        13032                5.622767                   4.0   

                    sum_cart_position  min_cart_position  max_cart_position  \
user_id product_id                                                            
1       196                    133206                  1                 60   
        10258                    8324                  1                 29   
        10326                   23160                  1                 37   
        12427                   30826                  1                 61   
        13032                   21091                  1                 66   

                    std_cart_position  aisle_reorder_ratio  \
user_id product_id                                           
1       196                  4.110813             0.638832   
        10258                3.567502             0.519170   
        10326                3.611700             0.718104   
        12427                4.782450             0.591986   
        13032                5.345184             0.571584   

                    department_reorder_ratio  reordered  
user_id product_id                                       
1       196                         0.653460        1.0  
        10258                       0.574180        1.0  
        10326                       0.649913        0.0  
        12427                       0.574180        0.0  
        13032                       0.560922        1.0  

[5 rows x 48 columns]

In [23]:
data_test.head()

times_brought  uxp_total_reordered  uxp_first_order_num  \
user_id product_id                                                            
3       248                     1                    0                    2   
        1005                    1                    0                   10   
        1819                    3                    2                    4   
        7503                    1                    0                    3   
        8021                    1                    0                    2   

                    uxp_avg_cart_pos  uxp_min_cart_pos  uxp_max_cart_pos  \
user_id product_id                                                         
3       248                 3.000000                 3                 3   
        1005                5.000000                 5                 5   
        1819                2.666667                 1                 5   
        7503                6.000000                 6                 6   
        8021                5.000000                 5                 5   

                    uxp_median_cart_pos  uxp_last_order_num  uxp_avg_dow  \
user_id product_id                                                         
3       248                         3.0                   2     3.000000   
        1005                        5.0                  10     3.000000   
        1819                        2.0                   7     0.666667   
        7503                        6.0                   3     3.000000   
        8021                        5.0                   2     3.000000   

                    uxp_median_dow  ...  prod_tot_reorders  \
user_id product_id                  ...                      
3       248                    3.0  ...               2550   
        1005                   3.0  ...                204   
        1819                   0.0  ...               1193   
        7503                   3.0  ...               6905   
        8021                   3.0  ...              16472   

                    prod_reordered_ratio  avg_cart_position  \
user_id product_id                                            
3       248                     0.400251          10.620782   
        1005                    0.440605           9.498920   
        1819                    0.492162           9.287541   
        7503                    0.553551           9.547379   
        8021                    0.591157           8.822854   

                    median_cart_position  sum_cart_position  \
user_id product_id                                            
3       248                          9.0              67665   
        1005                         8.0               4398   
        1819                         7.0              22513   
        7503                         8.0             119094   
        8021                         7.0             245840   

                    min_cart_position  max_cart_position  std_cart_position  \
user_id product_id                                                            
3       248                         1                 91           8.025390   
        1005                        1                 74           7.984800   
        1819                        1                 68           7.441465   
        7503                        1                 79           7.622521   
        8021                        1                 90           7.211007   

                    aisle_reorder_ratio  department_reorder_ratio  
user_id product_id                                                 
3       248                    0.519170                  0.574180  
        1005                   0.527615                  0.653460  
        1819                   0.487633                  0.346721  
        7503                   0.519170                  0.574180  
        8021                   0.528005                  0.402178  

[5 rows x 47 columns]

# Splitting data into train and test split

In [50]:
from sklearn.model_selection import train_test_split
from downcast import reduce


# replace Nan values with mean values
data_train.fillna(data_train.mean(), inplace=True)
data_train_reduced = reduce(data_train)

# Creating X and y(target variable)
X = data_train.drop(['reordered'], axis=1)
y = data_train['reordered']

In [51]:
X.head()

times_brought  uxp_total_reordered  uxp_first_order_num  \
user_id product_id                                                            
1       196                    10                    9                    1   
        10258                   9                    8                    2   
        10326                   1                    0                    5   
        12427                  10                    9                    1   
        13032                   3                    2                    2   

                    uxp_avg_cart_pos  uxp_min_cart_pos  uxp_max_cart_pos  \
user_id product_id                                                         
1       196                 1.400391                 1                 4   
        10258               3.333984                 2                 6   
        10326               5.000000                 5                 5   
        12427               3.300781                 1                 9   
        13032               6.332031                 5                 8   

                    uxp_median_cart_pos  uxp_last_order_num  uxp_avg_dow  \
user_id product_id                                                         
1       196                         1.0                  10     2.500000   
        10258                       3.0                  10     2.554688   
        10326                       5.0                   5     4.000000   
        12427                       2.5                  10     2.500000   
        13032                       6.0                  10     2.666016   

                    uxp_median_dow  ...  prod_tot_reorders  \
user_id product_id                  ...                      
1       196                    2.5  ...              27791   
        10258                  3.0  ...               1389   
        10326                  4.0  ...               3603   
        12427                  2.5  ...               4797   
        13032                  3.0  ...               2465   

                    prod_reordered_ratio  avg_cart_position  \
user_id product_id                                            
1       196                     0.776367           3.722656   
        10258                   0.713867           4.277344   
        10326                   0.651855           4.191406   
        12427                   0.740723           4.761719   
        13032                   0.657227           5.621094   

                    median_cart_position  sum_cart_position  \
user_id product_id                                            
1       196                          2.0             133206   
        10258                        3.0               8324   
        10326                        3.0              23160   
        12427                        3.0              30826   
        13032                        4.0              21091   

                    min_cart_position  max_cart_position  std_cart_position  \
user_id product_id                                                            
1       196                         1                 60           4.109375   
        10258                       1                 29           3.568359   
        10326                       1                 37           3.611328   
        12427                       1                 61           4.781250   
        13032                       1                 66           5.343750   

                    aisle_reorder_ratio  department_reorder_ratio  
user_id product_id                                                 
1       196                    0.638672                  0.653320  
        10258                  0.519043                  0.574219  
        10326                  0.718262                  0.649902  
        12427                  0.591797                  0.574219  
        13032                  0.571777                  0.561035  

[5 rows x 47 columns]

In [52]:
def normalize(df):
    data = df.copy()
    for feature_name in data.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        data[feature_name] = (data[feature_name] - min_value) / (max_value - min_value)
    return data  

In [53]:
# Normalize the data in each column
# X = normalize(X)
print(X.shape)
X.head()

(8474661, 47)


times_brought  uxp_total_reordered  uxp_first_order_num  \
user_id product_id                                                            
1       196                    10                    9                    1   
        10258                   9                    8                    2   
        10326                   1                    0                    5   
        12427                  10                    9                    1   
        13032                   3                    2                    2   

                    uxp_avg_cart_pos  uxp_min_cart_pos  uxp_max_cart_pos  \
user_id product_id                                                         
1       196                 1.400391                 1                 4   
        10258               3.333984                 2                 6   
        10326               5.000000                 5                 5   
        12427               3.300781                 1                 9   
        13032               6.332031                 5                 8   

                    uxp_median_cart_pos  uxp_last_order_num  uxp_avg_dow  \
user_id product_id                                                         
1       196                         1.0                  10     2.500000   
        10258                       3.0                  10     2.554688   
        10326                       5.0                   5     4.000000   
        12427                       2.5                  10     2.500000   
        13032                       6.0                  10     2.666016   

                    uxp_median_dow  ...  prod_tot_reorders  \
user_id product_id                  ...                      
1       196                    2.5  ...              27791   
        10258                  3.0  ...               1389   
        10326                  4.0  ...               3603   
        12427                  2.5  ...               4797   
        13032                  3.0  ...               2465   

                    prod_reordered_ratio  avg_cart_position  \
user_id product_id                                            
1       196                     0.776367           3.722656   
        10258                   0.713867           4.277344   
        10326                   0.651855           4.191406   
        12427                   0.740723           4.761719   
        13032                   0.657227           5.621094   

                    median_cart_position  sum_cart_position  \
user_id product_id                                            
1       196                          2.0             133206   
        10258                        3.0               8324   
        10326                        3.0              23160   
        12427                        3.0              30826   
        13032                        4.0              21091   

                    min_cart_position  max_cart_position  std_cart_position  \
user_id product_id                                                            
1       196                         1                 60           4.109375   
        10258                       1                 29           3.568359   
        10326                       1                 37           3.611328   
        12427                       1                 61           4.781250   
        13032                       1                 66           5.343750   

                    aisle_reorder_ratio  department_reorder_ratio  
user_id product_id                                                 
1       196                    0.638672                  0.653320  
        10258                  0.519043                  0.574219  
        10326                  0.718262                  0.649902  
        12427                  0.591797                  0.574219  
        13032                  0.571777                  0.561035  

[5 rows x 47 columns]

In [25]:
#Splitting dataset into train and test split
X.fillna(X.mean(), inplace=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10,stratify=y)

In [26]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(5932262, 47)
(5932262,)
(2542399, 47)
(2542399,)


# Feature Importance(Top 30)

In [65]:
# import pdb
# from sklearn.ensemble import RandomForestClassifier
# def get_imp_features(data,data_y,features):
#     clf = RandomForestClassifier(n_estimators=100,n_jobs=-1,verbose=10)
#     clf.fit(data,data_y)
#     imp_feature_indexes = np.argsort(clf.feature_importances_)[::-1]
#     top_30_indexes = imp_feature_indexes[:30]
#     feature_imp_values = np.take(clf.feature_importances_,top_30_indexes)
#     feature_imp_names = np.take(features,top_30_indexes)
#     return feature_imp_values,feature_imp_names,top_30_indexes

In [27]:
import pdb
from sklearn.ensemble import RandomForestClassifier

In [28]:
clf = RandomForestClassifier(n_estimators=100,n_jobs=-1,verbose=10)
clf.fit(X_train,y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.


building tree 1 of 100building tree 2 of 100
building tree 3 of 100
building tree 4 of 100

building tree 5 of 100
building tree 6 of 100building tree 7 of 100

building tree 8 of 100
building tree 9 of 100building tree 10 of 100
building tree 11 of 100building tree 12 of 100


building tree 13 of 100


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  2.8min


building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100


[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:  2.9min


...

building tree 86 of 100
building tree 87 of 100
building tree 88 of 100
building tree 89 of 100
building tree 90 of 100
building tree 91 of 100
building tree 92 of 100
building tree 93 of 100
building tree 94 of 100
building tree 95 of 100
building tree 96 of 100
building tree 97 of 100
building tree 98 of 100
building tree 99 of 100


[Parallel(n_jobs=-1)]: Done  88 out of 100 | elapsed: 23.6min remaining:  3.2min


building tree 100 of 100


[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 25.2min finished


RandomForestClassifier(n_jobs=-1, verbose=10)

In [44]:
feature_scores = pd.Series(clf.feature_importances_, index=X_train.columns).sort_values(ascending=False)

In [59]:
X_train.columns[25]

('mean_days_since_prior_order', '')

In [61]:
plt.subplots(figsize=(30, 24))
ax = sns.barplot(x=feature_scores, y=feature_scores.index, X_train)
ax.set_title("Visualize feature scores of the features")
ax.set_yticklabels(feature_scores.index)
ax.set_xlabel("Feature importance score")
ax.set_ylabel("Features")
plt.show()

SyntaxError: positional argument follows keyword argument (<ipython-input-61-377967119569>, line 2)

In [66]:
features = X_train.columns.tolist()

In [67]:
features

['times_brought',
 'uxp_total_reordered',
 'uxp_first_order_num',
 'uxp_avg_cart_pos',
 'uxp_min_cart_pos',
 'uxp_max_cart_pos',
 'uxp_median_cart_pos',
 'uxp_last_order_num',
 'uxp_avg_dow',
 'uxp_median_dow',
 'uxp_avg_hour_of_the_day',
 'uxp_median_hour_of_the_day',
 'uxp_order_rate',
 'uxp_sum_cart_pos',
 'uxp_min_order_days',
 'uxp_max_order_days',
 'uxp_median_order_days',
 'uxp_reorder_ratio',
 ('order_number', 'min'),
 ('order_number', 'max'),
 ('order_number', 'nanmean'),
 ('order_number', 'count'),
 ('max_day_of_week', ''),
 ('max_order_hour_of_day', ''),
 ('sum_days_since_prior_order', ''),
 ('mean_days_since_prior_order', ''),
 ('reordered_ratio', ''),
 ('total_num_orders', ''),
 ('user_total_prods', ''),
 ('user_unique_prods', ''),
 ('user_average_basket', ''),
 ('user_order_starts_at', ''),
 ('avg_no_prds_each_purchase', ''),
 ('median_no_prds_each_purchase', ''),
 ('min_no_prds_each_purchase', ''),
 ('max_no_prds_each_purchase', ''),
 'times_purchased',
 'prod_tot_reorde

In [49]:
clf = RandomForestClassifier(n_estimators=100,n_jobs=-1,verbose=10)
clf.fit(X_train,y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100building tree 4 of 100

building tree 5 of 100
building tree 6 of 100building tree 7 of 100
building tree 8 of 100

building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  2.7min


building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100


[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:  2.8min


...

building tree 86 of 100
building tree 87 of 100
building tree 88 of 100
building tree 89 of 100
building tree 90 of 100
building tree 91 of 100
building tree 92 of 100
building tree 93 of 100
building tree 94 of 100
building tree 95 of 100
building tree 96 of 100
building tree 97 of 100
building tree 98 of 100
building tree 99 of 100


[Parallel(n_jobs=-1)]: Done  88 out of 100 | elapsed: 54.2min remaining:  7.4min


building tree 100 of 100


[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 55.7min finished


RandomForestClassifier(n_jobs=-1, verbose=10)

In [80]:
imp_feature_indexes = np.argsort(clf.feature_importances_)[::-1]

In [81]:
top_30_indexes = imp_feature_indexes[:30]

In [82]:
top_30_indexes 

array([12, 30, 38, 31, 26, 25, 39, 44, 27, 24, 29, 37, 36, 41, 13, 43, 32,
       10,  7, 28, 21,  2, 45, 11,  3, 35,  1,  8,  5, 16], dtype=int64)

In [83]:
feature_imp_values = np.take(clf.feature_importances_,top_30_indexes)

In [84]:
feature_imp_names = np.take(features,top_30_indexes)

In [85]:
feature_imp_names

array(['uxp_order_rate', ('user_average_basket', ''),
       'prod_reordered_ratio', ('user_order_starts_at', ''),
       ('reordered_ratio', ''), ('mean_days_since_prior_order', ''),
       'avg_cart_position', 'std_cart_position', ('total_num_orders', ''),
       ('sum_days_since_prior_order', ''), ('user_unique_prods', ''),
       'prod_tot_reorders', 'times_purchased', 'sum_cart_position',
       'uxp_sum_cart_pos', 'max_cart_position',
       ('avg_no_prds_each_purchase', ''), 'uxp_avg_hour_of_the_day',
       'uxp_last_order_num', ('user_total_prods', ''),
       ('order_number', 'count'), 'uxp_first_order_num',
       'aisle_reorder_ratio', 'uxp_median_hour_of_the_day',
       'uxp_avg_cart_pos', ('max_no_prds_each_purchase', ''),
       'uxp_total_reordered', 'uxp_avg_dow', 'uxp_max_cart_pos',
       'uxp_median_order_days'], dtype=object)

In [51]:
imp_feature_indexes

array([12, 30, 38, 31, 26, 25, 39, 44, 27, 24, 29, 37, 36, 41, 13, 43, 32,
       10,  7, 28, 21,  2, 45, 11,  3, 35,  1,  8,  5, 16, 33, 17,  6,  0,
       15, 19, 14, 34,  4,  9, 46, 20, 22, 23, 40, 42, 18], dtype=int64)

In [74]:
top_30_indexes = imp_feature_indexes[:30]

In [75]:
feature_imp_values = np.take(clf.feature_importances_,top_30_indexes)
feature_imp_names = np.take(features,top_30_indexes)

In [76]:
feature_imp_values

array([0.06699192, 0.0329291 , 0.03276947, 0.03106602, 0.03055082,
       0.03012523, 0.02769216, 0.02758954, 0.02672317, 0.02669749,
       0.02550118, 0.02517609, 0.02477973, 0.02471582, 0.0242271 ,
       0.02419366, 0.02367016, 0.02346558, 0.02291885, 0.02270649,
       0.02223997, 0.02168641, 0.02077331, 0.02053976, 0.02040282,
       0.02039749, 0.01882398, 0.01864627, 0.01846334, 0.01844428])

In [77]:
feature_imp_names

array(['uxp_order_rate', ('user_average_basket', ''),
       'prod_reordered_ratio', ('user_order_starts_at', ''),
       ('reordered_ratio', ''), ('mean_days_since_prior_order', ''),
       'avg_cart_position', 'std_cart_position', ('total_num_orders', ''),
       ('sum_days_since_prior_order', ''), ('user_unique_prods', ''),
       'prod_tot_reorders', 'times_purchased', 'sum_cart_position',
       'uxp_sum_cart_pos', 'max_cart_position',
       ('avg_no_prds_each_purchase', ''), 'uxp_avg_hour_of_the_day',
       'uxp_last_order_num', ('user_total_prods', ''),
       ('order_number', 'count'), 'uxp_first_order_num',
       'aisle_reorder_ratio', 'uxp_median_hour_of_the_day',
       'uxp_avg_cart_pos', ('max_no_prds_each_purchase', ''),
       'uxp_total_reordered', 'uxp_avg_dow', 'uxp_max_cart_pos',
       'uxp_median_order_days'], dtype=object)

In [78]:
plt.figure(figsize = (10, 5))
ax = sns.barplot(x = feature_imp_names, y = feature_imp_values)
ax.set_xticklabels(labels = feature_imp_names, rotation = 90)
sns.set_palette(reversed(sns.color_palette("husl", 10)), 10)
plt.title('Feature Importance')
plt.xlabel('Feature Names')
plt.ylabel('Feature Importance Values')
plt.show()

ValueError: must supply a tuple to get_group with multiple grouping keys

<Figure size 720x360 with 0 Axes>

In [63]:
sns.barplot??

In [44]:
data_y

NameError: name 'data_y' is not defined

In [31]:
features = X_train.columns.tolist()
feature_imp_values,feature_imp_names,top_30_indexes = get_imp_features(X_train,y_train, features)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.


building tree 1 of 100building tree 2 of 100

building tree 3 of 100building tree 4 of 100

building tree 5 of 100
building tree 6 of 100building tree 7 of 100
building tree 8 of 100building tree 9 of 100

building tree 10 of 100
building tree 11 of 100

building tree 12 of 100


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  2.7min


building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100


[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:  2.9min


...

[Parallel(n_jobs=-1)]: Done  74 tasks      | elapsed: 67.2min


building tree 87 of 100
building tree 88 of 100
building tree 89 of 100
building tree 90 of 100
building tree 91 of 100
building tree 92 of 100
building tree 93 of 100
building tree 94 of 100
building tree 95 of 100
building tree 96 of 100
building tree 97 of 100
building tree 98 of 100
building tree 99 of 100


[Parallel(n_jobs=-1)]: Done  88 out of 100 | elapsed: 70.4min remaining:  9.6min


building tree 100 of 100


[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 71.9min finished


In [41]:
top_30_indexes

array([12, 30, 38, 31, 26, 25, 27, 44, 39, 24, 29, 37, 13, 41, 36, 43, 32,
       10,  7, 28, 21,  2, 45, 11,  3, 35, 17,  1,  5,  8], dtype=int64)

In [57]:
plt.figure(figsize = (10, 5))
ax = sns.barplot(feature_imp_names, feature_imp_names)
ax.set_xticklabels(labels = feature_imp_names, rotation = 90)
plt.title('Feature Importance')
plt.xlabel('Feature Names')
plt.ylabel('Feature Importance Values')
plt.show()

TypeError: Neither the `x` nor `y` variable appears to be numeric.

<Figure size 720x360 with 0 Axes>

In [40]:
X_train.head()

,,times_brought,uxp_total_reordered,uxp_first_order_num,uxp_avg_cart_pos,uxp_min_cart_pos,uxp_max_cart_pos,uxp_median_cart_pos,uxp_last_order_num,uxp_avg_dow,uxp_median_dow,...,prod_tot_reorders,prod_reordered_ratio,avg_cart_position,median_cart_position,sum_cart_position,min_cart_position,max_cart_position,std_cart_position,aisle_reorder_ratio,department_reorder_ratio
user_id,product_id,,,,,,,,,,,,,,,,,,,,,
130569,30984,1,0,5,12.0,12,12,12.0,5,5.000000,5.0,...,57,0.407227,10.289062,7.5,1440,1,63,9.210938,0.504395,0.457520
87031,31338,1,0,9,1.0,1,1,1.0,9,4.000000,4.0,...,2859,0.441895,9.515625,7.0,61546,1,82,7.933594,0.487549,0.346680
54106,10549,1,0,5,6.0,6,6,6.0,5,2.000000,2.0,...,239,0.387939,9.515625,7.0,5861,1,51,7.968750,0.392822,0.346680
76437,42625,5,4,10,19.0,7,39,16.0,18,1.400391,1.0,...,5865,0.438477,8.945312,7.0,119683,1,86,6.988281,0.516602,0.649902
17194,35851,1,0,10,15.0,15,15,15.0,10,4.000000,4.0,...,2365,0.645508,8.960938,7.0,32837,1,73,7.179688,0.686523,0.669922
